In [11]:
from dash import Dash, html, dcc
import dash_leaflet as dl
from dash.dependencies import Input, Output
from pyproj import CRS
from owslib.ogcapi.features import Features
import json

# Initialize the Dash app
app = Dash(__name__)

# Define the OpenStreetMap basemap layer
osm_layer = dl.BaseLayer(
    dl.TileLayer(
        url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
        maxZoom=19,
        attribution="© OpenStreetMap"
    ),
    name="OpenStreetMap",
    checked=True
)

# Define the Esri imagery basemap layer
esri_layer = dl.BaseLayer(
    dl.TileLayer(
        url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        maxZoom=19,
        attribution="© Esri"
    ),
    name="Esri Imagery",
    checked=False
)

# Define the Climate Stations overlay layer
climate_stations_layer = dl.Overlay(
    dl.WMSTileLayer(
        url="https://geo.weather.gc.ca/geomet-climate/?service=WMS&amp;version=1.3.0&amp;request=GetMap",
        layers="CLIMATE.STATIONS",
        format="image/png",
        transparent=True,
        attribution="Climate Stations",
        opacity=0.7  # Optional: Set opacity for the WMS layer
    ),
    name="Climate Stations",
    checked=True
)

# Create the map with the defined layers
vancouver_map = dl.Map(
    id="map-vancouver",
    style={'width': '100%', 'height': '80vh'},
    center=[49.2827, -123.1207],  # Coordinates for Vancouver
    zoom=10,
    children=[
        dl.LayersControl(
            [
                osm_layer,
                esri_layer,
                climate_stations_layer
            ],
            id="lc"
        )
    ]
)

# Define the Calgary map
calgary_map = dl.Map(
    id="map-calgary",
    style={'width': '100%', 'height': '100vh'},
    center=[51.0447, -114.0719],  # Coordinates for Calgary
    zoom=10,
    children=[
        dl.TileLayer(
            url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
            maxZoom=19,
            attribution="&copy; <a href='http://www.openstreetmap.org/copyright'>OpenStreetMap</a>"
        ),
        dl.WMSTileLayer(
            url="https://geo.weather.gc.ca/geomet-climate/?service=WMS&version=1.3.0&request=GetMap",
            layers="CLIMATE.STATIONS",
            format="image/png",
            transparent=True,
            attribution="Climate Stations",
            opacity=0.7  # Optional: Set opacity for the WMS layer
        )
    ]
)

# Define the app layout
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Vancouver', children=[
            vancouver_map,
            html.Pre(id='bbox-output'),  # Added here
            html.Div(id="log")
        ]),
        dcc.Tab(label='Calgary', children=[calgary_map])
    ])
])

def get_epsg_code(lat, lon):
    crs_wgs84 = CRS("EPSG:4326")
    utm_zone = int((lon + 180) // 6) + 1
    hemisphere = 'north' if lat >= 0 else 'south'
    crs_utm = CRS.from_proj4(f"+proj=utm +zone={utm_zone} +datum=NAD83 +units=m +no_defs")
    epsg_code = crs_utm.to_epsg()
    return epsg_code

@app.callback(
    Output('bbox-output', 'children'),
    Input('map-vancouver', 'bounds')
)
def update_bbox(bounds):
    if bounds is None:
        return "No bounds available"

    south_west = bounds[0]
    north_east = bounds[1]
    bbox = [south_west[1], south_west[0], north_east[1], north_east[0]]

    # Retrieve stations data
    oafeat = Features("https://api.weather.gc.ca/")
    station_data = oafeat.collection_items("climate-stations", bbox=bbox)

    # Verification of the retrieved data
    if "features" in station_data:
        station_data = json.dumps(station_data, indent=4)
    else:
        station_data = "No stations were found. Please verify the coordinates."

    return station_data

# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode='external')  # This works in Google Colab


Dash app running on http://127.0.0.1:8050/
